## 講座2.2 Pythonによるデータ分析入門

#### 共通事前処理

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# 共通事前処理

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.2f}'.format

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

### 欠損値対応

#### データ読み込み

In [ ]:
# 分析対象データCSV
df1_fn = 'l02_02_bridge.csv'

# 分析対象データをdfに代入
df1 = pd.read_csv(df1_fn)

# 先頭5行の表示
display(df1.head())

#### 欠損値確認

In [ ]:
# 欠損値を含めてMATERIALの値ごとの件数をカウント
df1['MATERIAL'].value_counts(dropna=False)

#### 欠損値除去

In [ ]:
# MATERIALの欠損値を行ごと削除
df2 = df1.dropna(subset=['MATERIAL'])

# 欠損値を含めてMATERIALの値ごとの件数をカウント
df2['MATERIAL'].value_counts(dropna=False)

#### 値補填

In [ ]:
# MATERIALの欠損値を'STEEL'で補填
df3 = df1.fillna({'MATERIAL':'STEEL'})

# 欠損値を含めてMATERIALの値ごとの件数をカウント
df3['MATERIAL'].value_counts(dropna=False)

### スケーリング

#### データ読み込み・確認

In [ ]:
# 分析対象データCSV
df4_fn = 'l02_02_bank-j.csv'

# データ読み込み
df4 = pd.read_csv(df4_fn)

# 先頭5行の表示
display(df4[['残高']].head().T)

# 残高の統計値確認
display(df4[['残高']].describe().T.iloc[:,1:])

#### 標準化

In [ ]:
# StandardScalerのインポート
from sklearn.preprocessing import StandardScaler

# StandardScalerインスタンスの生成
sc1 = StandardScaler()

df5 = df4.copy()
# 項目「残高」に標準化をかける
df5['残高'] = sc1.fit_transform(df5[['残高']])

# 先頭5行の表示
display(df5[['残高']].head().T)

# 残高の統計値確認
display(df5[['残高']].describe().T.iloc[:,1:])


#### 正規化

In [ ]:
# MinMaxScalerのインポート
from sklearn.preprocessing import MinMaxScaler

# MinMaxScalerインスタンスを範囲[0, 1]で生成
sc2 = MinMaxScaler(feature_range=(0, 1), copy=True)

df6 = df4.copy()
# 「残高」に正規化をかける
df6['残高'] = sc2.fit_transform(df6[['残高']])

# 先頭5行の表示
display(df6[['残高']].head().T)

# 残高の統計値確認
display(df6[['残高']].describe().T.iloc[:,1:])


### エンコーディング

#### データ読み込み・確認

In [ ]:
# 分析対象データCSV
df7_fn = 'l02_02_bank-j.csv'

# データ読み込み
df7 = pd.read_csv(df7_fn)

# 先頭5行の表示
display(df7[['学歴']].head().T)

# 学歴のカテゴリ値確認
display(df7[['学歴']].value_counts())


#### ラベルエンコーディング

In [ ]:
# LabelEncoderのインポート
from sklearn.preprocessing import LabelEncoder

# LabelEncoderのインスタンス生成
le = LabelEncoder()

df8 = df7.copy()
# ラベルエンコーディングの実施
df8['学歴_ENC'] = le.fit_transform(df8['学歴'])

# 結果確認
display(df8[['学歴', '学歴_ENC']].head())

#### ワン・ホット・エンコーディング

In [ ]:
df9 = df7.copy()

# df9: 項目「学歴」のみ抽出
df9 = df9[['学歴']]

# df10: 項目「学歴」をone hot encoding
df10 = pd.get_dummies(df9, columns=['学歴'])

# df11: 比較のため、2つのデータフレームを横連結
df11 = pd.concat([df9, df10], axis=1)

# 結果の先頭5行を確認
display(df11.head())

### 回帰モデルの精度評価

#### データ準備

In [ ]:
df12_fn = 'l02_02_day-j.csv'
df12 = pd.read_csv(df12_fn, parse_dates=['日付'])

# 「季節」「曜日」「天気」列をカテゴリ変数に変換
df12[['季節','曜日','天気']] = df12[['季節','曜日','天気']].astype('category')

# 目的変数 y を「登録ユーザー利用数」に設定
y = df12['登録ユーザー利用数']

# 説明変数 X を指定された項目を除いたデータフレームとして設定
X = df12.drop(['日付', '登録ユーザー利用数', '臨時ユーザー利用数', '全体ユーザー利用数'], axis=1)

# データを訓練用とテスト用に分割するための基準日を設定
split_date = pd.Timestamp("2012-11-01")
train_dates, test_dates = (df12['日付'] < split_date), (df12['日付'] >= split_date)

# 訓練用データとテスト用データに分割
X_train, X_test = X[train_dates], X[test_dates]
y_train, y_test = y[train_dates], y[test_dates]

# lightgbm回帰モデルのインポート
from lightgbm import LGBMRegressor

# lightgbm回帰モデルのインスタンスを作成
model = LGBMRegressor(num_threads=4, verbose=-1)

# モデルの学習を実行（scikit-learn互換のfit関数を使用）
model.fit(X_train, y_train)

# 構築したモデルを用いてテストデータ X_test に対する予測を実施
y_pred = model.predict(X_test)

$$
\begin{align*}
&\text{MSE} = \frac{1}{\text{N}}\sum_{k=1}^\text{N} (\widehat{y}^{(k)} - y^{(k)})^2\\
&\text{RMSE} = \sqrt{\text{MSE}}
\end{align*}
$$

#### RMSE(直接計算)

In [ ]:
# 平均値・標準偏差の算出
y_mean, y_std = y_test.mean(), y_test.std()

# 定義式に基づくMSEの計算
mse = ((y_test-y_pred)**2).mean()

# 定義式に基づくRMSEの計算
rmse = np.sqrt(mse)

# 結果確認
print(f'平均: {y_mean:.02f} 標準偏差: {y_std:.02f} MSE: {mse:.02f} RMSE: {rmse:.02f}')

#### RMSE(ライブラリ利用)

In [ ]:
from sklearn.metrics import mean_squared_error

# RMSEの計算
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# 結果確認
print(f'MSE: {mse:.02f} RMSE: {rmse:.02f}')

#### R2値

In [ ]:
from sklearn.metrics import r2_score

# R2値の計算
r2 = r2_score(y_test, y_pred)

# 結果確認
print(f'r2: {r2:.03f}')

### 分類モデルの精度評価

#### データ準備

In [ ]:
df13_fn = 'l02_02_bank-j.csv'

# CSVファイルを読み込み、データフレーム変数 df13 に代入
df13 = pd.read_csv(df13_fn)

# 「直前接触日」列の値が -1 のものを 9999 に置き換える
df13['直前接触日'] = df13['直前接触日'].replace(-1, 9999)

# 「月」列の英語表記を1から12の数字に置き換える
month_mapping = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}
df13['月'] = df13['月'].map(month_mapping)

# 数値項目以外のデータ型をカテゴリ型に変換
numeric_columns = df13.select_dtypes(include='number').columns
difference_columns = df13.columns.difference(numeric_columns)
df13[difference_columns] = df13[difference_columns].astype('category')

from sklearn.preprocessing import LabelEncoder
# カテゴリ型の列のみをラベルエンコーディング
label_encoders = {}
for column in df13.select_dtypes(include='category').columns:
    le = LabelEncoder()
    df13[column] = le.fit_transform(df13[column])

# 説明変数Xと目的変数yに分離し、通話時間をXから除外
X = df13.drop(columns=['申込有無', '通話時間'])  # 通話時間と申込有無を除外してXを作成
y = df13['申込有無']  # 目的変数yを作成

from sklearn.model_selection import train_test_split
# 説明変数Xと目的変数yを4:1の比率で分割し、乱数シードを123に設定
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

from lightgbm import LGBMClassifier
# LightGBM分類モデルを構築
# モデルを初期化。num_threads=4、メッセージ出力をオフ
model = LGBMClassifier(num_threads=4, verbose=-1)
model.fit(X_train, y_train)  # モデルの学習

# 構築したモデルを用いて、テストデータ X_test に対する予測を実施
y_pred = model.predict(X_test)  # 予測結果を y_pred に代入

#### 混同行列表示

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# 混同行列を作成
cm = confusion_matrix(y_test, y_pred)

# 混同行列をヒートマップ表示
plt.figure(figsize=(3, 3))
# ヒートマップの描画
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
    xticklabels=["非申込", "申込"], yticklabels=["非申込", "申込"])
plt.xlabel("予測値")
plt.ylabel("正解値")
plt.title("混同行列")

# グラフを表示
create_pdf()
plt.show()

#### 適合率・再現率・F値の計算

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# クラス1（申込あり）に対して、適合率、再現率、F値を算出
precision = precision_score(y_test, y_pred, pos_label=1)
recall = recall_score(y_test, y_pred, pos_label=1)
f1 = f1_score(y_test, y_pred, pos_label=1)

print(f'適合率: {precision:.03f} 再現率:{recall:.03f} F値:{f1:.03f}')

#### PR曲線の描画

In [ ]:
from sklearn.metrics import precision_recall_curve

# クラス1に対する予測確率を算出
y_pred_proba = model.predict_proba(X_test)[:, 1]  # クラス1の確率を取得

# クラス1に対するPR曲線のデータを取得
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba, pos_label=1)

# PR曲線を描画
plt.figure(figsize=(4, 4))
plt.plot(recall, precision, label="PR曲線 (クラス1)")
plt.xlabel("再現率 (Recall)")
plt.ylabel("適合率 (Precision)")
plt.title("PR曲線 (クラス1: 申込あり)")
plt.legend(loc="lower left")
plt.grid()
plt.show()

#### ROC曲線の描画

In [ ]:
from sklearn.metrics import roc_curve

# クラス1に対する予測確率を算出
y_pred_proba = model.predict_proba(X_test)[:, 1]  # クラス1の確率を取得

# ROC曲線のデータを取得
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba, pos_label=1)

# ROC曲線を描画
plt.figure(figsize=(4, 4))
plt.plot(fpr, tpr, label="ROC曲線 (クラス1)")
plt.plot([0, 1], [0, 1], 'k--', label="ランダム予測")
plt.xlabel("偽陽性率 (FPR)")
plt.ylabel("真陽性率 (TPR)")
plt.title("ROC曲線 (クラス1: 申込あり)")
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

# ROC曲線下の面積（AUC）を算出
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'ROC AUC:{roc_auc:.03f}')

### EXCEL生成

In [ ]:
df4[['残高']].head().T.to_excel('table-A02-02-04-1.xlsx')
df4[['残高']].describe().T.iloc[:,1:].to_excel('table-A02-02-04-2.xlsx')

In [ ]:
df5[['残高']].head().T.to_excel('table-A02-02-05-1.xlsx')
df5[['残高']].describe().T.iloc[:,1:].to_excel('table-A02-02-05-2.xlsx')

In [ ]:
df6[['残高']].head().T.to_excel('table-A02-02-06-1.xlsx')
df6[['残高']].describe().T.iloc[:,1:].to_excel('table-A02-02-06-2.xlsx')

In [ ]:
df7[['学歴']].head().T.to_excel('table-A02-02-07-1.xlsx')

In [ ]:
df7[['学歴']].value_counts().to_excel('table-A02-02-07-2.xlsx')

In [ ]:
df8[['学歴', '学歴_ENC']].head().to_excel('table-A02-02-08.xlsx')

In [ ]:
df11.head().to_excel('table-A02-02-09.xlsx')

In [ ]:
# PDF印刷用
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

try:
    from google.colab import files
except:
    pass

pdf_ind = 2
pdf_base = 'pdf-01-'

def create_pdf():
    global pdf_ind, pdf_base
    fn = f'{pdf_base}{pdf_ind:02d}.pdf'
    print(fn)
    plt.rcParams['figure.subplot.bottom'] = 0.3
    plt.rcParams['figure.subplot.left'] = 0.3
    plt.savefig(fn)
    try:
        files.download(fn)
    except:
        pass
    pdf_ind = pdf_ind + 1